In [2]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import urllib.request
import urllib.parse
import re
import requests
import csv
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from konlpy.tag import Twitter
from collections import Counter


base_url = 'https://www.teamblind.com'


topic = {'Military_story' : '/kr/topics/%EA%B5%B0%EB%8C%80%EC%9D%B4%EC%95%BC%EA%B8%B0',
'Recommended_code_and_sequence' :'/kr/topics/%EC%B6%94%EC%B2%9C%EC%BD%94%EB%93%9C%C2%B7%ED%94%84%EB%A6%AC%ED%80%80%EC%8B%9C',
'Blah_Blah' : '/kr/topics/%EB%B8%94%EB%9D%BC%EB%B8%94%EB%9D%BC',
'Stock_and_investment' : '/kr/topics/%EC%A3%BC%EC%8B%9D%C2%B7%ED%88%AC%EC%9E%90',
'love' : '/kr/topics/%EC%8D%B8%C2%B7%EC%97%B0%EC%95%A0',
'Cryptocurrency' : '/kr/topics/%EC%95%94%ED%98%B8%ED%99%94%ED%8F%90',
'Changing_jobs_and_Career' : '/kr/topics/%EC%9D%B4%EC%A7%81%C2%B7%EC%BB%A4%EB%A6%AC%EC%96%B4',
'traveling_and_eating' : '/kr/topics/%EC%97%AC%ED%96%89%C2%B7%EB%A8%B9%EB%B0%A9',
'real_estate' : '/kr/topics/%EB%B6%80%EB%8F%99%EC%82%B0',
'office_life' : '/kr/topics/%ED%9A%8C%EC%82%AC%EC%83%9D%ED%99%9C',
'hobby_life' : '/kr/topics/%EC%A7%81%EC%9E%A5%EC%9D%B8-%EC%B7%A8%EB%AF%B8%EC%83%9D%ED%99%9C',
'Car' : '/kr/topics/%EC%9E%90%EB%8F%99%EC%B0%A8',
'Shopping' : '/kr/topics/%EC%A7%80%EB%A6%84%C2%B7%EC%87%BC%ED%95%91',
'Sports' : '/kr/topics/%EC%8A%A4%ED%8F%AC%EC%B8%A0',
'companion_animals' : '/kr/topics/%EB%B0%98%EB%A0%A4%EB%8F%99%EB%AC%BC',
'Covid_19' : '/kr/topics/%EC%BD%94%EB%A1%9C%EB%82%98-%EB%B0%94%EC%9D%B4%EB%9F%AC%EC%8A%A4',
'Diet' : '/kr/topics/%ED%97%AC%EC%8A%A4%C2%B7%EB%8B%A4%EC%9D%B4%EC%96%B4%ED%8A%B8',
'parenting' : '/kr/topics/%EC%9C%A1%EC%95%84',
'Humor' : '/kr/topics/%EC%9C%A0%EC%9A%B0%EB%A8%B8',
'married_life' : '/kr/topics/%EA%B2%B0%ED%98%BC%EC%83%9D%ED%99%9C',
'Fashion' : '/kr/topics/%ED%8C%A8%EC%85%98%C2%B7%EB%B7%B0%ED%8B%B0',
'TV' : '/kr/topics/TV%C2%B7%EC%97%B0%EC%98%88'}


In [3]:
def drive(url):
    driver = webdriver.Chrome('/users/kong/chromedriver')
    driver.implicitly_wait(1)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return driver, soup

In [4]:
def topic_url_crawling(tp):
    contents_url = []
    url=base_url+topic.get(tp)
    driver, temp = drive(url)
    last_height = 0
    n=0
    while n<2000:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(1) 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") 
        time.sleep(1) 
        new_height = driver.execute_script("return document.body.scrollHeight") 
        if new_height == last_height: 
            break
        n+=1
    time.sleep(1.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    for div in soup.select('.name'):
        contents_url.append(div.findAll('a',href=True)[0]['href'])
    driver.close()
    return contents_url



In [5]:
car_url=topic_url_crawling('Car')

df=pd.DataFrame({'data': car_url})

df.to_csv('textminingurl.csv')

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=87.0.4280.88)


In [10]:
def page_crawling2(url_set):
    title=[]
    contents = []
    company=[]
    company_url=[]
    headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    n=0
    save=11
    for url in url_set:
        new_url=base_url+url
        req = requests.get(new_url, headers = headers)
        data=req.text
        soup = BeautifulSoup(data, 'html.parser')
        n+=1
        if n > 2000:
            n=0
            data = pd.DataFrame({'title_job' : title, 'contents_job':contents,'company_job':company,'company_url_job':company_url})
            data.to_excel('blind_data'+str(save)+'.xlsx')          
        if not soup.select('.not-found') and not url=="#":
            for div in soup.select('.article-view-head'):
                title.append(div.findAll('h2')[0].text)
            contents.append(soup.select('.contents-txt')[0].text)
            div1=soup.select('.name')[0]
            if div1.findAll('a',href=True):
                company.append(div1.text)
                company_url.append(div1.findAll('a',href=True)[0]['href'])
            else:
                company.append(div1.text)
                company_url.append('non')
    return title, contents, company, company_url

In [11]:
import csv    
f = open('zzz.csv', 'r', encoding='utf-8', newline='')
f.close()

In [12]:
df=pd.read_csv("zzz.csv")

In [13]:
df=(df[:1000])

In [14]:
df

,Unnamed: 0,data
0,0,/kr/post/%ED%8A%B8%EB%A0%88%EC%9D%BC%EB%B8%94%...
1,1,/kr/post/%EC%97%AC%EC%9E%90-%EC%B2%AB%EC%B0%A8...
2,2,/kr/post/%EB%82%B4%EB%85%84%EC%97%B0%EB%A7%90%...
3,3,/kr/post/%EC%A4%91%EA%B3%A0%EC%B0%A8-%EB%AD%90...
4,4,/kr/post/%EB%A0%89%EC%84%9C%EC%8A%A4-%EC%8B%A0...
...,...,...
995,995,/kr/post/GLE-vs-X56-vs-%EC%B9%B4%EC%9D%B4%EC%9...
996,996,/kr/post/%EC%8F%98%EB%A0%8C%ED%86%A0-%ED%95%98...
997,997,/kr/post/%EB%82%BC-%EC%83%88%EC%B0%A8%EA%B0%80...
998,998,/kr/post/%ED%8F%AC%EB%A5%B4%EC%89%90-%ED%83%80...


In [15]:
data=page_crawling2((df['data']))

KeyboardInterrupt: 

In [353]:
df=pd.DataFrame({'data': data})

df.to_csv('test1.csv')

In [354]:
# 데이터 변수에 title,text,company,suburl 열을 추가한 dataFrame을 생성한다.
data = pd.DataFrame(
    {'title': data[0],
     'text': data[1],
     'company': data[2],
     'suburl': data[3]})

In [355]:
# 값이 잘 들어갔나 확인
data

,title,text,company,suburl
0,트레일블레이저 vs 셀토스,"2500~2700 수준으로그리고 산다면장기렌트 vs 할부(선수금 30%, 개인사업자...",새회사\n · 닭****\n,non
1,여자 첫차 트랙스 중고 1890만원 어떨까요?,삼십대 여자 첫차로 중고 트랙스보고 있어요.주말에만 탈거고 디자인은 얘한테 꽂혀버려...,11번가\n · s*********\n,/kr/company/11%EB%B2%88%EA%B0%80/
2,내년연말에 외제차 사구싶은데추천좀해둉,안뇽 난 곧 30살되눈 으냉원이구 여자얌연봉은 7천-8천사이일한지 5년됫구 부동산제...,NH농협은행\n · l********\n,/kr/company/NH%EB%86%8D%ED%98%91%EC%9D%80%ED%9...
3,중고차 뭐살까요.,유지비 및 등등 조언좀 얻고싶네요 마세라티 같은경유 유지하는데 특별히 많이 들어가는...,LS엠트론\n · p******\n,/kr/company/LS%EC%97%A0%ED%8A%B8%EB%A1%A0/
4,렉서스 신형 너무 이쁜거 같아,진짜 너무 너무 너무 사고 싶은데 중고가도 쎄서 사면 카푸어 당첨이라 차마 못사네 ...,SK이노베이션\n · 1*********\n,/kr/company/SK%EC%9D%B4%EB%85%B8%EB%B2%A0%EC%9...
...,...,...,...,...
987,GLE vs. X5/6 vs. 카이엔,"안녕 블라형들 👋이번에 한국 귀국하면서 미국서 몰던 렉서스 RX를 정리하고, 한국서...",한국오라클\n · n*******\n,/kr/company/%ED%95%9C%EA%B5%AD%EC%98%A4%EB%9D%...
988,쏘렌토 하브 또는 셀토스 중고 고민입니다.,아이 한명 있구요~ 첫차라 고민이네요. 어떤차 사는게 좋을까요 쏘렌토같이 큰차 잘 ...,NEOPLE\n · ******\n,/kr/company/NEOPLE/
989,낼 새차가 올 예정!,"저번주에 계약하고, 오늘 등록했어!오늘 썬팅이랑 유리막 다하고 내일 집으로 탁송받을...",LG전자\n · w*****\n,/kr/company/LG%EC%A0%84%EC%9E%90/
990,포르쉐 타이칸 인증 200km대 실화?,"전기차는 테슬라 따라가는 회사가 현대기아밖에 없는듯,폭스바겐 ID4도 양산 밀렸고h...",새회사\n · b*****\n,non


In [356]:
# 닉네임을 제거한 후 회사 이름을 유지
for i in range(len(data['company'])):
    data['company'][i]=data['company'][i].split('\n')[0][1:]
 
# 본문에 있는 영어를 소문자로 변경 -> Bmw BMW 등 다양한 명칭이 있어 Carlist와 매칭이 안될 수 있음
company_total=[]
for i in range(len(data['company'])):
    company_total.append(data['company'][i])
    data['text'][i]=data['text'][i].lower()
 
# 본문에 추가한 모든 회사중 딕셔너리를 이용하여 Key값은 회사 , Value값은 등장한 빈도수로 표현함
dict_company=dict()
for i in range(len(data['company'])):
    if data['company'][i] in company_total:
        dict_company.setdefault(data['company'][i] , company_total.count(data['company'][i]))


In [1016]:
# 값이 잘 들어갔나 확인
print(dict_company)

{'새회사': 92, '11번가': 2, 'NH농협은행': 4, 'SK이노베이션': 7, 'LG화학': 19, '공무원': 66, 'NCSOFT': 9, '현대자동차': 13, '한국철도공사': 24, '국방과학연구소': 2, '두산인프라코어': 2, 'SGS': 2, '한국MSD': 2, '한국수력원자력': 5, '현대건설': 6, '동일알루미늄': 2, '한화': 5, '종근당': 3, '코리아에프티': 3, '한국콜마': 3, '기업은행': 7, '서울특별시교육청': 6, '캐논코리아': 7, '현대모비스': 3, 'LG전자': 16, '국민건강보험공단': 5, 'LG디스플레이': 17, 'LG CNS': 8, '한국오라클': 2, 'SK하이닉스': 13, 'LG생활건강': 3, 'CJ올리브영': 2, '제일약품': 2, '삼성중공업': 3, '이마트': 7, 'NH농협': 3, '고려저축은행': 5, '대우건설': 2, '경찰청': 8, '현대엘리베이터': 7, '서울시설공단': 2, '스타트업': 21, '삼성전자': 38, 'NAVER': 11, '웰크론한텍': 18, '썸에이지': 2, '에스엘': 3, '한국팜비오': 2, '한국전력공사': 5, '롯데케미칼': 3, '대한항공': 10, 'CJ제일제당': 5, 'DB': 2, '카카오': 10, '삼성SDS': 7, '코오롱인더스트리': 5, '스타벅스': 4, '롯데정보통신': 5, '서울특별시': 3, 'LG유플러스': 2, '라인플러스': 5, '오스템임플란트': 5, '롯데GRS': 2, '대림산업': 4, '포스코케미칼': 2, '우리은행': 7, 'COUPANG': 8, '셀트리온': 3, 'Apple Korea': 2, '새마을금고': 4, '서울교통공사': 11, '현대트랜시스': 2, '코레일로지스': 2, '아시아나항공': 11, '삼일회계법인': 3, 'DB손해보험': 5, '대구도시철도공사': 2, '롯데쇼핑e커머스사업본부': 2, '이스타항공': 2, '위메프': 2, '

In [1017]:

car_list='아토스 클릭 베르나 아반테 아반떼 쏘나타 티뷰론 그랜저 118d 외제차 에쿠스 싼타모 겔로퍼 포르쉐 페라리 람보르기니 마이바흐 롤스로이드 아우디 폭스바겐 테라칸 산타페 스타렉스 트라제 그레이스 다이너스티 투스카니 라비타 옵티마 프라이드 아벨라 엘란 크레도스 스펙트라 카렌스 비스토 리오 포텐샤 엔터프라이즈 스포티지 카니발 카스타 레토나 슈마 타우너 라이노 세피아 세레스 베스타 복사 라세티 티코 칼로스 다마스 라보 마티즈 르망 씨에로 에스페로 라노스 누비라 레간자 프린스 브러엄 아카디아 매그너스 레조 무쏘 코란도 체어맨 이스타나 SM520 야무진 클리오 SM3 SM5 SM7 QM3 QM5 칼리스타 체어맨뉴 체어맨 체어맨H 체어맨H 클래식 체어맨w 체어맨 카이런 액티언 거화 지프 렉스턴 슈퍼렉스턴 렉스턴W 무쏘 코란도 훼미리 코란도 코란도 티볼리 로디우스 코란도 투리스모 무쏘 액티언 코란도 닛산디젤트럭 DA트럭 SY트럭 이스타나 MCI 고속버스 버스 에어로 트랜스타 옵티마 마젠티스 리오 포르테 쎄라토 스펙트라 슈마쿱 카덴자 피간토 론도 쿠오리스 세도나 VQ KX5 보레고 조이스 아만티 클라우스 상트로 아토스 엑센트 포니 란트라 소니카 엘란트라 라비타 매트릭스 하이웨이 밴 트라제 GL8 크레타 JM ix35 투싼 ix35 싼타페 클래식 ix55 XG 센티니얼 아제라 겟츠 쿠페 티뷰론 i10 i20 HB20 i25 i35 i45 엘란트라 투어링 엘란트라 GT c클 s클 로헨스 로헨스 벤츠 e클 쏠라리스 베르나 H100 iMAX H350 HD120 HD160 테슬라 전기차 벤츠S 벤츠A 벤츠C bmw'
# 다른 좋은 방법 있으면 추천좀 해주세요..

# 문자열을 스플릿 함수를 이용하여 공백을 기준으로 리스트로 변경해줌
car_list=car_list.split(' ')

# 중복된 차량 이름이 있을 수 있으므로 데이터구조 set을 이용하여 중복제거
car_list=set(car_list)
car_list=list(car_list)
 

In [1018]:
# data['title']를 차량리스트와 매칭시켜 언급된 차량의 모델과 회사이름을 출력
for i in range(len(data['title'])):
    for j in range(len(car_list)):
        if car_list[j] in data['title'][i]:
            print(data['title'][i] + ' : ' + car_list[j] ,'\n ', data['company'][j])
            print('')
            dict_company.setdefault(data['company'][j] , car_list[j].count(car_list[j]))

내년연말에 외제차 사구싶은데추천좀해둉 : 외제차 
  인팩

카니발 역수입 가능? 한가요? : 카니발 
  앰코코리아

투싼 NX4 색상 : 투싼 
  SK이노베이션

체어맨 조아??? : 체어맨 
  삼성중공업

중고차 고민..(벤츠c클 vs lf소나타 하브) : 벤츠 
  공무원

중고차 고민..(벤츠c클 vs lf소나타 하브) : c클 
  고려저축은행

쏘나타 냉간시 시동꺼짐 + 엔진부조 : 쏘나타 
  서울특별시교육청

투싼 하브 신차구매 서비스 추천 부탁!  : 투싼 
  SK이노베이션

혹시 폭스바겐 타시는분 있나요? : 폭스바겐 
  현대그린푸드

Dr.bmw 행사 안하나요? : bmw 
  웰크론한텍

[단독] 화재·브레이크 결함...애증의 코나 전기차 국내 단종 : 전기차 
  제일약품

테슬라 겨울에 밖에 차 세워두면 안열리나봐? : 테슬라 
  새회사

아우디 a7 55tfsi 사려는데 : 아우디 
  새회사

nx4 투싼 샀는데...이거 어떻게 해야되? : 투싼 
  SK이노베이션

초짜배기 공뭔인데 외제차 끌어도되냐? : 외제차 
  인팩

외제차 중고가격 감가는 얼마로 보면 적절할까요? : 외제차 
  인팩

현재 케파 타는데 벤츠 a클래스 세단 옮기는거 추천하시나요 : 벤츠 
  공무원

아반떼 디자인 변천사 : 아반떼 
  금융투자협회

아우디 실화에여?? : 아우디 
  새회사

벤츠 GLB 220 or 250 4M 딜러 소개 : 벤츠 
  공무원

테슬라3 어떤가요? : 테슬라 
  새회사

엄마차 벤츠 폐차 -> 니로로 사주려는데 : 벤츠 
  공무원

투싼 아마존 그레이 직접 보신 분? : 투싼 
  SK이노베이션

와 그랜저 사이드미러 교체 43만원 나옴ㅋㅋㅋ : 그랜저 
  한국콜마

5500 GV70  vs  6400 벤츠e250 : 벤츠 
  공무원

bmw320i 견적알아보고싶습니다 : bmw 
  웰크론한텍

에쿠스 380 2013년식 얼마 받을수 있을까?🤔 : 에쿠스 
  한국철도공사

BMW 6GT 견적 내주

In [1020]:
# data['text']를 차량리스트와 매칭시켜 언급된 차량의 모델과 회사이름을 출력
for i in range(len(data['text'])):
    for j in range(len(car_list)):
        if car_list[j] in data['text'][i]:
            print(data['text'][i] + ' : ' + car_list[j] ,'\n ', data['company'][j])
            print('')
            dict_company.setdefault(data['company'][j] , car_list[j].count(car_list[j]))

            

안뇽 난 곧 30살되눈 으냉원이구 여자얌연봉은 7천-8천사이일한지 5년됫구 부동산제태크해서 자산은 한 4억졍두될거같애..근데5년동안 중고차타구잇어서내년연말에 외제차로바꾸려하거든예산은 5천만원?(올현금쓸거얌)벤츠c클이나... 뭐쪼매난거사고싶은데아무런 개념이읍서ㅜㅜ 머사면돼 : 외제차 
  인팩

안뇽 난 곧 30살되눈 으냉원이구 여자얌연봉은 7천-8천사이일한지 5년됫구 부동산제태크해서 자산은 한 4억졍두될거같애..근데5년동안 중고차타구잇어서내년연말에 외제차로바꾸려하거든예산은 5천만원?(올현금쓸거얌)벤츠c클이나... 뭐쪼매난거사고싶은데아무런 개념이읍서ㅜㅜ 머사면돼 : 벤츠 
  공무원

안뇽 난 곧 30살되눈 으냉원이구 여자얌연봉은 7천-8천사이일한지 5년됫구 부동산제태크해서 자산은 한 4억졍두될거같애..근데5년동안 중고차타구잇어서내년연말에 외제차로바꾸려하거든예산은 5천만원?(올현금쓸거얌)벤츠c클이나... 뭐쪼매난거사고싶은데아무런 개념이읍서ㅜㅜ 머사면돼 : c클 
  고려저축은행

진짜 너무 너무 너무 사고 싶은데 중고가도 쎄서 사면 카푸어 당첨이라 차마 못사네 후ㅠㅠ 타협해서 벤츠c클 중고로 가기로 맘먹었지먀 진짜 이 디자인은 내 취향에 너무 맞는 디자인..bmw7세대, 벤츠s클, 아우디 a8, 재규어xj 등 고급차 옆좌석 여러번 타봤지만 렉서스es300h가 안정적인 주행감과 정숙성에서 다 발랐었음(하이브리드라 당연한건가..?) : 벤츠 
  공무원

진짜 너무 너무 너무 사고 싶은데 중고가도 쎄서 사면 카푸어 당첨이라 차마 못사네 후ㅠㅠ 타협해서 벤츠c클 중고로 가기로 맘먹었지먀 진짜 이 디자인은 내 취향에 너무 맞는 디자인..bmw7세대, 벤츠s클, 아우디 a8, 재규어xj 등 고급차 옆좌석 여러번 타봤지만 렉서스es300h가 안정적인 주행감과 정숙성에서 다 발랐었음(하이브리드라 당연한건가..?) : s클 
  서울시설공단

진짜 너무 너무 너무 사고 싶은데 중고가도 쎄서 사면 카푸어 당첨이라 차마 못사네 후ㅠㅠ 타협해서 벤츠c클 중고로 가기로 맘먹었지먀 진

보증기간 1년 정도 남은 외제차를 중고로 구매하려고 합니다.보증 끝난 외제차 기준으로 감가 30% 가격을 생각하면 될까요?차종의 인기에 따라 영향을 많이 받는 것으로 보입니다만원하는 차종을 얼마 선에 구매해야 지혜롭게 구입한 수준이 될지 판단이 안서네요지금 보는 차량은 머스탱 2.3 에코 부스트 쿠페입니다. : 쿠페 
  기업은행

보증기간 1년 정도 남은 외제차를 중고로 구매하려고 합니다.보증 끝난 외제차 기준으로 감가 30% 가격을 생각하면 될까요?차종의 인기에 따라 영향을 많이 받는 것으로 보입니다만원하는 차종을 얼마 선에 구매해야 지혜롭게 구입한 수준이 될지 판단이 안서네요지금 보는 차량은 머스탱 2.3 에코 부스트 쿠페입니다. : 외제차 
  인팩

첫차로 케파산지 1년 좀 안됫구 어차피 첫차라 오래 안탈예정으로 트림도 낮게하고 타는 기한은 한 3년 정도 예상했어요!예전부터 꿈꾸던 드림카가 벤츠인데 에이클래스 세단 가격이 나쁘지않고 프로모션 받고 하면 괜찮을거같은데 이십대중반 여성이 끌기 괜찮을까요?아니면 조금 더 길게보고 나중에 더 좋은차를 사는게 나을까요? 바꾼다면 빠르면 1년내로 길어도 2년 안에 바꿀예정입니다. : 벤츠 
  공무원

중고포함해서 추돌시 안전한차가 어떤건가요?27살이고 연봉은 4300입니다모은돈은 4천정도되고집얘기하실까봐 말하는데 결혼할때 부모님이 5억정도 해주신데요.주말만 거의타고 왕복 130km 고향이나 여자친구랑 여행다녀요.10년된 아반떼로 고속도로 타는데 가끔 무섭더라구요..ㅊㅊ좀 부탁드립니다. : 아반떼 
  금융투자협회

2013년도에 운전면허증 취득하고 지금까지 한번 운전했는데 이번에 투싼 중고로 구매하려합니다무리일까용?!!투싼 무리면 아반떼 생각하는데.. 투싼 타고시푼뎅 ㅠㅠㅠ 장농이 운전하기 힘들까용? : 투싼 
  SK이노베이션

2013년도에 운전면허증 취득하고 지금까지 한번 운전했는데 이번에 투싼 중고로 구매하려합니다무리일까용?!!투싼 무리면 아반떼 생각하는데.. 투싼 타고시푼뎅 ㅠㅠㅠ 장농이 운전하기 힘들

30초에 결혼계획은 없습니다 여자친구도 없어요ㅠㅜ코로나시국에 대중교통타고 서울이랑 회사자취방 왔다갔다하는것도 무섭고해서, 차를 하나 사려고 합니다(평생 뚜벅이었읍니다..)저는 옛날부터 가끔 유튭에서 교통사고영상을 많이봐서(고속도로 사고영상)...경차는 안살거구 제목에 쓰여진것처럼 새개중에 고르려고 합니다1. 평일은 통근버스타서 운전할일 거의없음2. 주말에 집-서울 갈때 자차탈거임3. 운전완전초보고 대부분 혼자탈것으로 예상4. 한번사면 5년이상 탈것임5. 좀 안전한차였으면 좋겠는데 솔직히 저 세개 다 일반적으로 보는 교통사고 안전성은 다 비슷하다 보면되겠죠...?현재 통장잔고가 3500정도라 취등록세해서 3200이 max고 나머지 남는돈으론 주식하려합니다추천부탁드려요 고수님둘 : 버스 
  현대모비스

외제차는 프로모션할때프로모션외에 딜러할인 같은거 있나요? 딜러할인 없이 그냥 공식프로모션만 적용되서 구매하나요? : 외제차 
  인팩

대출은 분양권때문이구차종은 그랜저나 아테온? (지금은 10년된 아방)20대 후반이야! : 그랜저 
  한국콜마

저번에도 투표로 물어봤던놈이야 ㅎㅎ거기선 투싼35% 쏘렌토 65%였음!투싼하브랑 쏘렌토 터보(비슷한가격선3700) 고민중인디고민중인게1.남자 혼자서 쏘렌토 끄는데 필요이상으로 좀 크다는 느낌은 안들까? 너무 아빠차같은 느낌이 나진 않을까?2.230마력의 투싼과 280마력의 쏘렌토 크기차이랑 휠베이스 생각하면 주행감은 비슷할까?(추후 둘다 시승신청할 계획이긴함)3.연 2만키로도 안타는데 하이브리드가 굳이 필요할까?4.옵션의 투싼이냐 차급의 쏘렌토냐!형들 진지하게 조언부탁해ㅜㅜ : 투싼 
  SK이노베이션

데일리로 출퇴근을 하는 용이고 차 잘나가는거 좋아하는 스타일이라서 사려고 하는데 어떨까?(일년에 만킬로 언더로 운행해서 디젤/하브는 오바인가 같아서 말야)싼타페를 원래 사려고 했는데 실내가 이전 모델 같아서 소렌토로 생각 중이거든 : 싼타페 
  삼성SDS

중고로 차를 바꾸고싶은데 차량 가격 4000선에서 구매할 생각

싼타페 tm 이랑 더 뉴 싼타페랑 다른거야 같은거야?또 a/l 이랑 f/l이랑 뭐가 차이 나는거야?? : 싼타페 
  삼성SDS

싼타페 한대 뽑으려구 하는데내가 차를 1도 몰라서 말야....차도 핸드폰 마냥 지점이랑 사람마다 조건이 엄청 다르고 그래?? 몇백씩 차이가 나는거야?질문이 이상하긴 한데 싸게 구할 수 있는 팁이 있어?신차 뽑아야해서. : 싼타페 
  삼성SDS

아버지께서 신차 구매하신다고 하셔서 문의드립니다.투싼 하이브리드 인스퍼레이션멀티미디어 네비2현대 스마트센스2가지 추가하신다고 합니다! : 투싼 
  SK이노베이션

이번에 이직을 하면서 대중교통이 좀 불편해서출퇴근용 차를 사려고 합니다예산은 대충 3,500언더로 잡고있고 출퇴근 거리는 편도 24키로 나온답니다!아반떼, 아반떼 하이브리드일단은 이렇게 고민중인데요하이브리드가 엄청 조용하더라구여근데 차량가격도 있고 출퇴근 이외에 많이 타고다닐까 생각도 들어서 굳이 하이브리드일 필요가 있나싶기도 하고..하이브리드 배터리가 노후로 인해서 4년? 정도면 슬슬 교체가 필요하다고 하는것같더라구요혹시 다른 차도 추천해주실만한게 있을까요? : 아반떼 
  금융투자협회

캠리 가솔린 흰색 vs 쏘나타 n라인 인스퍼레이션 흰색 vs 그랜저 2.5 가솔린 캘리그래피 흰색 : 그랜저 
  한국콜마

캠리 가솔린 흰색 vs 쏘나타 n라인 인스퍼레이션 흰색 vs 그랜저 2.5 가솔린 캘리그래피 흰색 : 쏘나타 
  서울특별시교육청

kb차차차에 kb캐피탈꺼 장기렌트카 중고로올라왔는데요 싼타페 19000km 2500정도인데 장기렌트 중고는 안쳐다보는게 맞나여? : 싼타페 
  삼성SDS

초년생입니다ㅜ 호갱 당하기 싫어서 이 사이트 저 사이트 돌아다니며 찾고는 있는데 매물들이 다 너무 비싸네요 다들 어디서 중고차 사세요. 저는 아반떼 ad 5년 미만 키로수도 5만 미만으로 천만원 언저리에 사고싶은데 의외로 비싸네요 아직도 ...ㅠㅠㅠ : 아반떼 
  금융투자협회

쏘나타 n라인 나온것들 리뷰보니깐 8단 dct 조합이랑 

완전 차알못인데,제네시스랑 그랜져는 어떤 느낌으로 보면 돼?같은 현대여도 아예 배다른 자식같은거야? 부품도 다른거 쓰고?외제차는 이번 생에선 그른거 같고...이제 30중-후로 들어서는데 좀 오래 탈 현기 블랙 세단으로 보려고 당장 살건 아니고 내년 상반기 중에는 뽑을거 같아.g80, g70, 그랜져, k7 이정도 놓고 비교중인데 가이드 좀 잡아주라!마음은 g70인데 가끔씩 뒤에 부모님도 태워드리고 하려면 좁겠지?ㅎㅎ : 외제차 
  인팩

전기차 아닌감? : 전기차 
  제일약품

아반떼 풀옵 2400주고 살만한건가? 가격이 적당한건가? 이번에 잘나와서 적당한건가 라면인건가 라면인건가요 : 아반떼 
  금융투자협회

버스,지하철 복잡하고 사람 닿는거 싫고 배차 기다리기 싫어서집앞 걸어가는게 더 빠른 마트 갈때도 차 이용출근 지하철보다 40분 더걸려도 차 이용퇴근 지하철보다 1시간 더걸려도 차 이용 한다네요 : 버스 
  현대모비스

아반떼 md 한국타이어 195 65r 15 추천좀1개당 10만이하러 무료장착 포함 : 아반떼 
  금융투자협회

차체제어시스템이 전륜 2륜에서도 필요한건가요? 효과가 있나요? 전륜이면 보통 언더스티어 나는거 같은데 구동력제어만으로 오버스티어는 몰라도 언더스티어를 잡아주나요? : 버스 
  현대모비스

안녕 우리 결혼한지 1년된 신혼부부인데계속 뚜벅이로 살다가 이제 중고차 사려구 하는데 예산은 1300만원 ~ 1700만원 정도야 내년에 임신 계획 있어서 소형 suv 구매할까 하고 코나, 니로, 스포티지, 셀토스 중에 고민중인데 가성비 연비 뭐가 제일 좋을까? 우린 평일에는 둘다 대중교통 이용하고주말에만 쓸거 같은데 이 가격에 중고차 사서 수리비 더 들면 할부로 새차 사는게 나을까?아님 중고차 사서 5년 정도 타고 새차 사는게 나을까?남편이 주로 운전할건데 너무 큰 차는 주차 걱정된다고 해서 코나 사고 싶다고 하는데 뒷자석이 좁다고 해서 걱정이네 ㅠㅠ둘다 차알못이라 다들 조언과 투표 부탁해 🙄 : 스포티지 
  스타트업

나도이제 차사

안녕 형님들인생 첫차로 중고차를 사보려 하는데미혼 30초bmw320d f302500 선에서 사려고 하는데어떻게 생각해?주 용도는 놀러다닐때 타는거구(운전은 잘해👏)5년정도  타다 결혼하면 새차 사거나 하면 좋을거 같은데형님들 생각은 어땡?.?돈은 지각 아래 맨틀층 수저라중고를 알아보는 거니얼마 얹아서 더 좋은차는 힘들엉ㅠ결혼후 집생각해서 2500까지만ㅜㅜ(연비 좋은차!가 좋아) : bmw 
  웰크론한텍

부모님한테 물려받을 유산 없고전세집 혼자 자취한다고 했을때요그래도 어느정도 최소라는게 있을거 같아서요일단월 400-450 벌면서레인지로버,bmw 5,벤츠 c클 e클타는 친구들 보면 무리 같긴하더라고요(일정 금액 일시불+60개월 할부 등) : 벤츠 
  공무원

부모님한테 물려받을 유산 없고전세집 혼자 자취한다고 했을때요그래도 어느정도 최소라는게 있을거 같아서요일단월 400-450 벌면서레인지로버,bmw 5,벤츠 c클 e클타는 친구들 보면 무리 같긴하더라고요(일정 금액 일시불+60개월 할부 등) : e클 
  새회사

부모님한테 물려받을 유산 없고전세집 혼자 자취한다고 했을때요그래도 어느정도 최소라는게 있을거 같아서요일단월 400-450 벌면서레인지로버,bmw 5,벤츠 c클 e클타는 친구들 보면 무리 같긴하더라고요(일정 금액 일시불+60개월 할부 등) : c클 
  고려저축은행

부모님한테 물려받을 유산 없고전세집 혼자 자취한다고 했을때요그래도 어느정도 최소라는게 있을거 같아서요일단월 400-450 벌면서레인지로버,bmw 5,벤츠 c클 e클타는 친구들 보면 무리 같긴하더라고요(일정 금액 일시불+60개월 할부 등) : bmw 
  웰크론한텍

30대 초반이고 서울살아.데이트 할때마다 차있으면 좋겠다라는 생각이 너무들어..ㅠㅠㅠㅠ 사면 후회할까?결혼하고 애갖으면 차사는게 제일 베스트라고는 하지만삶의질충 입장에서 보면차있으면 삶의질이 업그레이드 되는건 맞는것 같아..6억정도되는 아파트 하나에 따로 투자해둔 분양권하나있고재테크는 관심가지고 계속 하는중이야.대신 부채가

In [1021]:
dict_company.items()

dict_items([('새회사', 92), ('11번가', 2), ('NH농협은행', 4), ('SK이노베이션', 7), ('LG화학', 19), ('공무원', 66), ('NCSOFT', 9), ('현대자동차', 13), ('한국철도공사', 24), ('국방과학연구소', 2), ('두산인프라코어', 2), ('SGS', 2), ('한국MSD', 2), ('한국수력원자력', 5), ('현대건설', 6), ('동일알루미늄', 2), ('한화', 5), ('종근당', 3), ('코리아에프티', 3), ('한국콜마', 3), ('기업은행', 7), ('서울특별시교육청', 6), ('캐논코리아', 7), ('현대모비스', 3), ('LG전자', 16), ('국민건강보험공단', 5), ('LG디스플레이', 17), ('LG CNS', 8), ('한국오라클', 2), ('SK하이닉스', 13), ('LG생활건강', 3), ('CJ올리브영', 2), ('제일약품', 2), ('삼성중공업', 3), ('이마트', 7), ('NH농협', 3), ('고려저축은행', 5), ('대우건설', 2), ('경찰청', 8), ('현대엘리베이터', 7), ('서울시설공단', 2), ('스타트업', 21), ('삼성전자', 38), ('NAVER', 11), ('웰크론한텍', 18), ('썸에이지', 2), ('에스엘', 3), ('한국팜비오', 2), ('한국전력공사', 5), ('롯데케미칼', 3), ('대한항공', 10), ('CJ제일제당', 5), ('DB', 2), ('카카오', 10), ('삼성SDS', 7), ('코오롱인더스트리', 5), ('스타벅스', 4), ('롯데정보통신', 5), ('서울특별시', 3), ('LG유플러스', 2), ('라인플러스', 5), ('오스템임플란트', 5), ('롯데GRS', 2), ('대림산업', 4), ('포스코케미칼', 2), ('우리은행', 7), ('COUPANG', 8), ('셀트리온', 3), ('Apple Korea', 2), 

In [1022]:
dict_copy_compay=dict_company
cou=0

In [1023]:

dict_company.values()==1

False

In [1038]:
for i, j in dict_company.items():
    if j == 1:
        cou=cou+1
        dict_company.pop(i,1)
print("총" +str(len(dict_company))+'개 중에'+ str(cou)+ "개 삭제 했습니다.")


# 값이 1 을 가진 회사는 데이터 표본에 영향을 미치므로 삭제 error 있음 수정해야 함

총136개 중에9개 삭제 했습니다.


In [1039]:
dict_company.items()

dict_items([('새회사', 92), ('11번가', 2), ('NH농협은행', 4), ('SK이노베이션', 7), ('LG화학', 19), ('공무원', 66), ('NCSOFT', 9), ('현대자동차', 13), ('한국철도공사', 24), ('국방과학연구소', 2), ('두산인프라코어', 2), ('SGS', 2), ('한국MSD', 2), ('한국수력원자력', 5), ('현대건설', 6), ('동일알루미늄', 2), ('한화', 5), ('종근당', 3), ('코리아에프티', 3), ('한국콜마', 3), ('기업은행', 7), ('서울특별시교육청', 6), ('캐논코리아', 7), ('현대모비스', 3), ('LG전자', 16), ('국민건강보험공단', 5), ('LG디스플레이', 17), ('LG CNS', 8), ('한국오라클', 2), ('SK하이닉스', 13), ('LG생활건강', 3), ('CJ올리브영', 2), ('제일약품', 2), ('삼성중공업', 3), ('이마트', 7), ('NH농협', 3), ('고려저축은행', 5), ('대우건설', 2), ('경찰청', 8), ('현대엘리베이터', 7), ('서울시설공단', 2), ('스타트업', 21), ('삼성전자', 38), ('NAVER', 11), ('웰크론한텍', 18), ('썸에이지', 2), ('에스엘', 3), ('한국팜비오', 2), ('한국전력공사', 5), ('롯데케미칼', 3), ('대한항공', 10), ('CJ제일제당', 5), ('DB', 2), ('카카오', 10), ('삼성SDS', 7), ('코오롱인더스트리', 5), ('스타벅스', 4), ('롯데정보통신', 5), ('서울특별시', 3), ('LG유플러스', 2), ('라인플러스', 5), ('오스템임플란트', 5), ('롯데GRS', 2), ('대림산업', 4), ('포스코케미칼', 2), ('우리은행', 7), ('COUPANG', 8), ('셀트리온', 3), ('Apple Korea', 2), 

In [1040]:
for i , j in dict_company.items():
    print(i, j ,'명 총 ', round( j / len(data['text'] * 100),2 ) ,'% 센트 입니다.' )
    

새회사 92 명 총  0.09 % 센트 입니다.
11번가 2 명 총  0.0 % 센트 입니다.
NH농협은행 4 명 총  0.0 % 센트 입니다.
SK이노베이션 7 명 총  0.01 % 센트 입니다.
LG화학 19 명 총  0.02 % 센트 입니다.
공무원 66 명 총  0.07 % 센트 입니다.
NCSOFT 9 명 총  0.01 % 센트 입니다.
현대자동차 13 명 총  0.01 % 센트 입니다.
한국철도공사 24 명 총  0.02 % 센트 입니다.
국방과학연구소 2 명 총  0.0 % 센트 입니다.
두산인프라코어 2 명 총  0.0 % 센트 입니다.
SGS 2 명 총  0.0 % 센트 입니다.
한국MSD 2 명 총  0.0 % 센트 입니다.
한국수력원자력 5 명 총  0.01 % 센트 입니다.
현대건설 6 명 총  0.01 % 센트 입니다.
동일알루미늄 2 명 총  0.0 % 센트 입니다.
한화 5 명 총  0.01 % 센트 입니다.
종근당 3 명 총  0.0 % 센트 입니다.
코리아에프티 3 명 총  0.0 % 센트 입니다.
한국콜마 3 명 총  0.0 % 센트 입니다.
기업은행 7 명 총  0.01 % 센트 입니다.
서울특별시교육청 6 명 총  0.01 % 센트 입니다.
캐논코리아 7 명 총  0.01 % 센트 입니다.
현대모비스 3 명 총  0.0 % 센트 입니다.
LG전자 16 명 총  0.02 % 센트 입니다.
국민건강보험공단 5 명 총  0.01 % 센트 입니다.
LG디스플레이 17 명 총  0.02 % 센트 입니다.
LG CNS 8 명 총  0.01 % 센트 입니다.
한국오라클 2 명 총  0.0 % 센트 입니다.
SK하이닉스 13 명 총  0.01 % 센트 입니다.
LG생활건강 3 명 총  0.0 % 센트 입니다.
CJ올리브영 2 명 총  0.0 % 센트 입니다.
제일약품 2 명 총  0.0 % 센트 입니다.
삼성중공업 3 명 총  0.0 % 센트 입니다.
이마트 7 명 총  0.01 % 센트 입니다.
NH농협 3 명 총  0.0 % 센트 입니

In [1041]:
import openpyxl

wb = openpyxl.load_workbook('company_final.xlsx')
sheet = wb.active

company_pd = pd.read_excel('company_final.xlsx')

company_pd


,Unnamed: 0,company_name,company_rate,기업구분,"공기업, 공무원, 공단",비영리법인,대기업,중견기업,중소기업,"전문직(간호사, 의사, 법무법인, 회계사)",외국계,O==
0,0,한국서부발전,3.9,공기업,O,NaN,NaN,NaN,NaN,NaN,NaN,O
1,1,신세계인터내셔날,3.5,대기업,NaN,NaN,O,NaN,NaN,NaN,NaN,O
2,2,국토안전관리원,3.3,공기업,O,NaN,NaN,NaN,NaN,NaN,NaN,O
3,3,안랩,3.3,중견기업,NaN,NaN,NaN,O,NaN,NaN,NaN,O
4,4,EBS,3.5,비영리법인,NaN,O,NaN,NaN,NaN,NaN,NaN,O
...,...,...,...,...,...,...,...,...,...,...,...,...
790,정보검색,https://www.nicebizinfo.com/cm/CM0100M001GE.nice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
791,NaN,네이버 검색,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
792,참고사항,위의 사이트에서 국내 기업 현황에 나오는 건 기업구분으로 적었고 나오지 않는 외국기...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793,NaN,이름 기준으로 조회하여 정확도가 떨어질 수 있음 *유사한 이름의 기업이 다수 존재,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1042]:
company_pd['company_rate']

0      3.9
1      3.5
2      3.3
3      3.3
4      3.5
      ... 
790    NaN
791    NaN
792    NaN
793    NaN
794    NaN
Name: company_rate, Length: 795, dtype: float64

In [1043]:
# 필요 없는 열 삭제
company_pd = company_pd.drop("Unnamed: 0", axis=1)
company_pd = company_pd.drop("company_rate", axis=1)
company_pd = company_pd.drop("공기업, 공무원, 공단", axis=1)
company_pd = company_pd.drop("비영리법인", axis=1)
company_pd = company_pd.drop("대기업", axis=1)
company_pd = company_pd.drop("중견기업", axis=1)
company_pd = company_pd.drop("중소기업", axis=1)
company_pd = company_pd.drop("전문직(간호사, 의사, 법무법인, 회계사)", axis=1)
company_pd = company_pd.drop("외국계", axis=1)
company_pd = company_pd.drop("O==", axis=1)

In [1044]:
company_pd

,company_name,기업구분
0,한국서부발전,공기업
1,신세계인터내셔날,대기업
2,국토안전관리원,공기업
3,안랩,중견기업
4,EBS,비영리법인
...,...,...
790,https://www.nicebizinfo.com/cm/CM0100M001GE.nice,NaN
791,네이버 검색,NaN
792,위의 사이트에서 국내 기업 현황에 나오는 건 기업구분으로 적었고 나오지 않는 외국기...,NaN
793,이름 기준으로 조회하여 정확도가 떨어질 수 있음 *유사한 이름의 기업이 다수 존재,NaN


In [1]:
# 회사를 각 Class별로 구분함
company={'bigcompany' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '대기업'],
        'smallcompany' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '중소기업'] ,
        'official' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '공기업'] ,
        'specialist' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '전문직'] ,
        'globaloffice' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '외국계']}




# 왜 있어야 하는 데이터인지 잘 모르겠음, 삭제 예정
# company_list=[]
# for i in company.keys():
#     company_list = company_list+company[i]


NameError: name 'company_pd' is not defined

In [1050]:
bigcompany_len=len(company['bigcompany'])
smallcompany_len=len(company['smallcompany'])
official_len=len(company['official'])
specialist_len=len(company['specialist'])
globaloffice_len=len(company['globaloffice'])
print('블라인드 에서 대기업의 비율은 ' ,round(bigcompany_len/(bigcompany_len + smallcompany_len +official_len +specialist_len +globaloffice_len) *100,2) ,"%입니다." )
print('블라인드 에서 중소기업의 비율은 ' ,round(smallcompany_len/(bigcompany_len + smallcompany_len +official_len +specialist_len +globaloffice_len) *100,2) ,"%입니다." )
print('블라인드 에서 공공기관의 비율은 ' ,round(official_len/(bigcompany_len + smallcompany_len +official_len +specialist_len +globaloffice_len) *100,2) ,"%입니다." )
print('블라인드 에서 공공기관의 비율은 ' ,round(specialist_len/(bigcompany_len + smallcompany_len +official_len +specialist_len +globaloffice_len) *100,2) ,"%입니다." )
print('블라인드 에서 공공기관의 비율은 ' ,round(globaloffice_len/(bigcompany_len + smallcompany_len +official_len +specialist_len +globaloffice_len) *100,2) ,"%입니다." )

블라인드 에서 대기업의 비율은  50.94 %입니다.
블라인드 에서 중소기업의 비율은  24.18 %입니다.
블라인드 에서 공공기관의 비율은  19.95 %입니다.
블라인드 에서 공공기관의 비율은  1.64 %입니다.
블라인드 에서 공공기관의 비율은  3.29 %입니다.


In [1051]:
# 위에는 게시글 이옹자의 비율이다. 하지만 차량 카테고리 이용자의 비율은 어떻까?

# 회사를 각 Class별로 구분함
company={'car_bigcompany' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '대기업'],
        'car_smallcompany' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '중소기업'] ,
        'car_official' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '공기업'] ,
        'car_specialist' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '전문직'] ,
        'car_globaloffice' : [company_pd['company_name'][i] for i in range(len(company_pd['기업구분'])) if company_pd['기업구분'][i] == '외국계']}



# #
# company_list=[]
# for i in company.keys():
#     company_list = company_list+company[i]



In [ ]:
# 차 가격을 상, 중 , 하 로 나눠서 각 집단 그룹의 재직을 살펴볼 예정
high_value=[]
middle_value=[]
low_value=[]


In [284]:
# open으로 txt파일을 열고 read()를 이용하여 읽는다.
text = open('test.txt').read() 


twitter = Twitter()

# twitter함수를 통해 읽어들인 내용의 형태소를 분석한다.
sentences_tag = []
sentences_tag = twitter.pos(text) 

noun_adj_list = []


# tag가 명사이거나 형용사인 단어들만 noun_adj_list에 넣어준다.
for word, tag in sentences_tag:
    if tag in ['Noun' , 'Adjective']: 
        noun_adj_list.append(word)


# 가장 많이 나온 단어부터 40개를 저장한다.
counts = Counter(noun_adj_list)
tags = counts.most_common(40) 


# WordCloud를 생성한다.
# 한글을 분석하기위해 font를 한글로 지정해주어야 된다. macOS는 .otf , window는 .ttf 파일의 위치를
# 지정해준다. (ex. '/Font/GodoM.otf')
wc = WordCloud(font_path=[FontAwesome.otf],background_color="white", max_font_size=60)
cloud = wc.generate_from_frequencies(dict(tags))


# 생성된 WordCloud를 test.jpg로 보낸다.
cloud.to_file('test.jpg')

/Users/kong/opt/anaconda3/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


JVMNotFoundException: No JVM shared library file (libjli.dylib) found. Try setting up the JAVA_HOME environment variable properly.